In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/arc-prize-2024/arc-agi_training_solutions.json
/kaggle/input/arc-prize-2024/arc-agi_evaluation_solutions.json
/kaggle/input/arc-prize-2024/arc-agi_evaluation_challenges.json
/kaggle/input/arc-prize-2024/sample_submission.json
/kaggle/input/arc-prize-2024/arc-agi_training_challenges.json
/kaggle/input/arc-prize-2024/arc-agi_test_challenges.json


In [2]:
import json

training_solutions_path = '/kaggle/input/arc-prize-2024/arc-agi_training_solutions.json'
evaluation_solutions_path = '/kaggle/input/arc-prize-2024/arc-agi_evaluation_solutions.json'
evaluation_challenges_path = '/kaggle/input/arc-prize-2024/arc-agi_evaluation_challenges.json'
sample_submission_path = '/kaggle/input/arc-prize-2024/sample_submission.json'
training_challenges_path = '/kaggle/input/arc-prize-2024/arc-agi_training_challenges.json'
test_challenges_path = '/kaggle/input/arc-prize-2024/arc-agi_test_challenges.json'

#function to load JSON data
def load_json_data(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

#load each dataset
training_solutions = load_json_data(training_solutions_path)
evaluation_solutions = load_json_data(evaluation_solutions_path)
evaluation_challenges = load_json_data(evaluation_challenges_path)
sample_submission = load_json_data(sample_submission_path)
training_challenges = load_json_data(training_challenges_path)
test_challenges = load_json_data(test_challenges_path)

In [3]:
#inspect the structure of the data
def inspect_data(data, name):
    print(f"Inspecting {name}:")
    if isinstance(data, list):
        print(f"Number of items: {len(data)}")
        if len(data) > 0:
            print(f"Example item: {data[0]}")
    elif isinstance(data, dict):
        print(f"Number of keys: {len(data.keys())}")
        if len(data.keys()) > 0:
            first_key = list(data.keys())[0]
            print(f"Example item under key '{first_key}': {data[first_key]}")
    else:
        print("Unknown data type")
    print("\n")

inspect_data(training_solutions, "Training Solutions")
inspect_data(evaluation_solutions, "Evaluation Solutions")
inspect_data(evaluation_challenges, "Evaluation Challenges")
inspect_data(sample_submission, "Sample Submission")
inspect_data(training_challenges, "Training Challenges")
inspect_data(test_challenges, "Test Challenges")

Inspecting Training Solutions:
Number of keys: 400
Example item under key '007bbfb7': [[[7, 0, 7, 0, 0, 0, 7, 0, 7], [7, 0, 7, 0, 0, 0, 7, 0, 7], [7, 7, 0, 0, 0, 0, 7, 7, 0], [7, 0, 7, 0, 0, 0, 7, 0, 7], [7, 0, 7, 0, 0, 0, 7, 0, 7], [7, 7, 0, 0, 0, 0, 7, 7, 0], [7, 0, 7, 7, 0, 7, 0, 0, 0], [7, 0, 7, 7, 0, 7, 0, 0, 0], [7, 7, 0, 7, 7, 0, 0, 0, 0]]]


Inspecting Evaluation Solutions:
Number of keys: 400
Example item under key '00576224': [[[3, 2, 3, 2, 3, 2], [7, 8, 7, 8, 7, 8], [2, 3, 2, 3, 2, 3], [8, 7, 8, 7, 8, 7], [3, 2, 3, 2, 3, 2], [7, 8, 7, 8, 7, 8]]]


Inspecting Evaluation Challenges:
Number of keys: 400
Example item under key '00576224': {'test': [{'input': [[3, 2], [7, 8]]}], 'train': [{'input': [[8, 6], [6, 4]], 'output': [[8, 6, 8, 6, 8, 6], [6, 4, 6, 4, 6, 4], [6, 8, 6, 8, 6, 8], [4, 6, 4, 6, 4, 6], [8, 6, 8, 6, 8, 6], [6, 4, 6, 4, 6, 4]]}, {'input': [[7, 9], [4, 3]], 'output': [[7, 9, 7, 9, 7, 9], [4, 3, 4, 3, 4, 3], [9, 7, 9, 7, 9, 7], [3, 4, 3, 4, 3, 4], [7, 9, 7, 9, 7, 

In [4]:
def get_max_grid_size(challenges, solutions):
    max_input_height, max_input_width = 0, 0
    max_output_height, max_output_width = 0, 0
    
    for key in challenges.keys():
        challenge = challenges[key]
        for example in challenge['train']:
            input_grid = example['input']
            output_grid = example['output']
            max_input_height = max(max_input_height, len(input_grid))
            max_input_width = max(max_input_width, len(input_grid[0]))
            max_output_height = max(max_output_height, len(output_grid))
            max_output_width = max(max_output_width, len(output_grid[0]))
        for test_case in challenge['test']:
            test_input = test_case['input']
            max_input_height = max(max_input_height, len(test_input))
            max_input_width = max(max_input_width, len(test_input[0]))
            #assuming test_output size can be derived similarly

    return max_input_height, max_input_width, max_output_height, max_output_width

max_input_height, max_input_width, max_output_height, max_output_width = get_max_grid_size(training_challenges, training_solutions)
print(f"Max input size: {max_input_height}x{max_input_width}, Max output size: {max_output_height}x{max_output_width}")

Max input size: 30x30, Max output size: 30x30


In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class ARCDataset(Dataset):
    def __init__(self, challenges, solutions, max_size, transform=None):
        self.data = []
        self.max_size = max_size
        self.transform = transform
        for key in challenges.keys():
            challenge = challenges[key]
            solution = solutions[key]
            for example in challenge['train']:
                input_grid = example['input']
                output_grid = example['output']
                self.data.append((input_grid, output_grid))
            for test_case in challenge['test']:
                test_input = test_case['input']
                #use the corresponding solution as the target output
                test_output = solution[len(self.data) % len(solution)]
                self.data.append((test_input, test_output))

    def __len__(self):
        return len(self.data)

    def pad_grid(self, grid):
        padded_grid = np.zeros(self.max_size)
        for i in range(len(grid)):
            for j in range(len(grid[0])):
                padded_grid[i][j] = grid[i][j]
        return padded_grid

    def __getitem__(self, idx):
        input_grid, output_grid = self.data[idx]
        input_grid = self.pad_grid(input_grid)
        output_grid = self.pad_grid(output_grid)
        input_grid = torch.tensor(input_grid, dtype=torch.float32).unsqueeze(0)
        output_grid = torch.tensor(output_grid, dtype=torch.float32).unsqueeze(0)
        if self.transform:
            input_grid = self.transform(input_grid)
            output_grid = self.transform(output_grid)
        return input_grid, output_grid

#define the maximum size
max_size = (30, 30)

#create datasets with padding
transform = transforms.Compose([transforms.Lambda(lambda x: x)])  #no additional transform needed
training_dataset = ARCDataset(training_challenges, training_solutions, max_size, transform=transform)
evaluation_dataset = ARCDataset(evaluation_challenges, evaluation_solutions, max_size, transform=transform)

#create dataloaders
training_loader = DataLoader(training_dataset, batch_size=1, shuffle=True)
evaluation_loader = DataLoader(evaluation_dataset, batch_size=1, shuffle=False)

In [6]:
def preprocess_input(x, normalize=False, mean=0.5, std=0.5):
    """
    Preprocesses the input tensor `x` and returns it along with dimensions needed by subsequent modules.
    
    Args:
    - x (torch.Tensor): Input tensor with shape (batch_size, channels, height, width).
    - normalize (bool): Whether to normalize the input tensor.
    - mean (float or list): Mean for normalization (scalar or list for each channel).
    - std (float or list): Standard deviation for normalization (scalar or list for each channel).
    
    Returns:
    - x (torch.Tensor): Preprocessed input tensor.
    - input_channels (int): Number of channels in the input tensor.
    - input_height (int): Height of the input tensor.
    - input_width (int): Width of the input tensor.
    
    Raises:
    - ValueError: If the input tensor `x` does not have the expected number of dimensions.
    - ValueError: If the input tensor `x` does not have the expected number of channels for a module.
    """
    #ensure the input tensor has 4 dimensions (batch_size, channels, height, width)
    if x.dim() != 4:
        raise ValueError(f"Expected input tensor with 4 dimensions (batch_size, channels, height, width), got {x.dim()} dimensions instead.")
    
    #check the number of channels and adjust as needed
    input_channels = x.size(1)
    
    #ensure that the number of channels is 1 or 3 (grayscale or RGB)
    if input_channels not in [1, 3]:
        raise ValueError(f"Expected input tensor with 1 or 3 channels, got {input_channels} channels instead.")
    
    #extract input dimensions
    input_height = x.size(2)
    input_width = x.size(3)
    
    #normalize the input tensor if required
    if normalize:
        if isinstance(mean, list) and isinstance(std, list):
            if len(mean) != input_channels or len(std) != input_channels:
                raise ValueError("Mean and std lists must have the same length as the number of channels.")
            mean = torch.tensor(mean).view(1, input_channels, 1, 1)
            std = torch.tensor(std).view(1, input_channels, 1, 1)
        else:
            mean = torch.tensor([mean] * input_channels).view(1, input_channels, 1, 1)
            std = torch.tensor([std] * input_channels).view(1, input_channels, 1, 1)
        x = (x - mean) / std
    
    return x, input_channels, input_height, input_width

x1 = torch.randn(8, 1, 28, 28)  #example input tensor with shape (batch_size, channels, height, width)
x2 = torch.randn(8, 3, 32, 32)  #example input tensor with 3 channels (RGB)

#without normalization
preprocessed_x1, channels1, height1, width1 = preprocess_input(x1)
print(f"Preprocessed tensor shape: {preprocessed_x1.shape}")
print(f"Channels: {channels1}, Height: {height1}, Width: {width1}")

#with normalization
preprocessed_x2, channels2, height2, width2 = preprocess_input(x2, normalize=True, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
print(f"Preprocessed tensor shape: {preprocessed_x2.shape}")
print(f"Channels: {channels2}, Height: {height2}, Width: {width2}")

Preprocessed tensor shape: torch.Size([8, 1, 28, 28])
Channels: 1, Height: 28, Width: 28
Preprocessed tensor shape: torch.Size([8, 3, 32, 32])
Channels: 3, Height: 32, Width: 32


In [7]:
import torch.nn as nn
import torch.nn.functional as F

#perception module
class PerceptionModule(nn.Module):
    def __init__(self, in_channels):
        super(PerceptionModule, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        
        #use Adaptive Average Pooling to handle varying input sizes
        self.adaptive_pool = nn.AdaptiveAvgPool2d((1, 1))  #output size (1, 1)
        
        #fully connected layer
        self.fc = nn.Linear(64, 512)  #adjusted input size to 64 from adaptive pooling

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.adaptive_pool(x)
        x = x.view(x.size(0), -1)  #flatten the tensor
        x = self.fc(x)
        return x

In [8]:
#memory module
class MemoryModule(nn.Module):
    def __init__(self, input_size, output_size=512, normalize=False, mean=0.5, std=0.5):
        super(MemoryModule, self).__init__()
        self.normalize = normalize
        self.mean = mean
        self.std = std
        
        #define a fully connected layer with configurable input and output sizes
        self.fc = nn.Linear(input_size, output_size)
        
    def preprocess_input(self, x):
        if self.normalize:
            #normalize input if required
            x = (x - self.mean) / self.std
        return x

    def forward(self, x):
        #apply preprocessing
        x = self.preprocess_input(x)
        
        #ensure the input is of the correct shape
        x = x.view(x.size(0), -1)
        
        #forward pass through the fully connected layer
        x = self.fc(x)
        return x

In [9]:
#creativity module
class CreativityModule(nn.Module):
    def __init__(self, input_size, latent_size=128):
        super(CreativityModule, self).__init__()
        self.fc1 = nn.Linear(input_size, latent_size)
        self.fc2 = nn.Linear(latent_size, input_size)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [10]:
#reasoning module
class ReasoningModule(nn.Module):
    def __init__(self, input_size, output_size):
        super(ReasoningModule, self).__init__()
        self.fc1 = nn.Linear(input_size, 128) 
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, output_size)
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [11]:
#compassionate module
class CompassionateModule(nn.Module):
    def __init__(self, input_size):
        super(CompassionateModule, self).__init__()
        self.fc1 = nn.Linear(input_size, 128) 
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 128)  
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [12]:
#bias detection and mitigation module
class BiasDetectionMitigationModule(nn.Module):
    def __init__(self, input_size):
        super(BiasDetectionMitigationModule, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.fc3 = nn.Linear(128, 128) 
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [13]:
#planning module
class PlanningModule(nn.Module):
    def __init__(self, input_size):
        super(PlanningModule, self).__init__()
        self.fc = nn.Linear(input_size, 128) 
        
    def forward(self, x):
        x = self.fc(x)
        x = F.relu(x)  #apply ReLU activation function after linear transformation
        return x

In [14]:
#attention module
class AttentionModule(nn.Module):
    def __init__(self, input_size, output_size=128): 
        super(AttentionModule, self).__init__()
        self.fc_query = nn.Linear(input_size, output_size)
        self.fc_key = nn.Linear(input_size, output_size)
        self.fc_value = nn.Linear(input_size, output_size)
        self.softmax = nn.Softmax(dim=-1)
    
    def forward(self, x):
        batch_size = x.size(0)
        x = x.unsqueeze(1)  
        query = self.fc_query(x)
        key = self.fc_key(x)
        value = self.fc_value(x)

        attention_scores = torch.matmul(query, key.transpose(-2, -1)) / (query.size(-1) ** 0.5)
        attention_weights = self.softmax(attention_scores)
        context_layer = torch.matmul(attention_weights, value)

        return context_layer.squeeze(1)

In [15]:
#learning module
class LearningModule(nn.Module):
    def __init__(self, input_size, hidden_size1=512, hidden_size2=256):
        super(LearningModule, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, input_size) 
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [16]:
#meta-reasoning module
class MetaReasoningModule(nn.Module):
    def __init__(self, input_size, output_size):
        super(MetaReasoningModule, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, output_size)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [17]:
#communication module
class CommunicationModule(nn.Module):
    def __init__(self, input_size, output_size):
        super(CommunicationModule, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, output_size)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [18]:
#integration module
class IntegrationModule(nn.Module):
    def __init__(self, input_size):
        super(IntegrationModule, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 256)

    def forward(self, x):
        print(f"Integration input size: {x.size()}")
        x = F.relu(self.fc1(x))
        x = self.dropout(x)  #apply dropout after activation function
        x = F.relu(self.fc2(x))
        return x

In [19]:
import logging
import random  #for simulating health check status
import shutil   #for handling file operations

#resilience module
class ResilienceModule(nn.Module):
    def __init__(self, input_size, monitor_interval=100, log_file='system.log'):
        super(ResilienceModule, self).__init__()
        self.monitor_interval = monitor_interval
        self.log_file = log_file

        #initialize logger
        logging.basicConfig(filename=self.log_file, level=logging.INFO)

        #example network components
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 16)
        self.fc5 = nn.Linear(16, 1)

        #step counter for monitoring
        self.global_step = 0

        #directory to save model checkpoints
        self.checkpoint_dir = 'checkpoints'
        os.makedirs(self.checkpoint_dir, exist_ok=True)

        #initialize a placeholder for the last checkpoint path
        self.last_checkpoint_path = os.path.join(self.checkpoint_dir, 'last_checkpoint.pth')

    def forward(self, x):
        try:
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(x))
            x = F.relu(self.fc3(x))
            x = F.relu(self.fc4(x))
            x = self.fc5(x)

            #simulate monitoring every monitor_interval steps
            if self.training and self.global_step % self.monitor_interval == 0:
                self.monitor_system()

            return x
        except Exception as e:
            self.handle_error(e)
            #return a tensor of zeros in case of an error
            return torch.zeros(x.size(0), 1)

    def monitor_system(self):
        #simulate system health check
        system_health = self.check_system_health()
        logging.info(f"System health check at step {self.global_step}: {'Healthy' if system_health else 'Unhealthy'}")

        if not system_health:
            self.recover_system()

    def check_system_health(self):
        #simulate health checks
        try:
            #simulate checking CPU and Memory usage
            cpu_usage = random.uniform(0, 100)  #simulate CPU usage percentage
            memory_usage = random.uniform(0, 100)  #simulate memory usage percentage

            #simulate error rates or system load
            error_rate = random.uniform(0, 1)  #simulate some error rate

            #define thresholds for health check
            cpu_threshold = 80.0
            memory_threshold = 80.0
            error_threshold = 0.1  #example threshold for error rate

            #check system health
            healthy = (cpu_usage < cpu_threshold and 
                       memory_usage < memory_threshold and 
                       error_rate < error_threshold)

            #log the simulated metrics
            logging.info(f"CPU Usage: {cpu_usage:.2f}%")
            logging.info(f"Memory Usage: {memory_usage:.2f}%")
            logging.info(f"Error Rate: {error_rate:.2f}")

            return healthy
        except Exception as e:
            logging.error(f"Health check failed: {e}")
            return False

    def recover_system(self):
        logging.warning("Initiating system recovery process.")
        try:
            #example: rollback to the last checkpoint
            self.rollback_update()
            logging.info("System recovery process completed.")
        except Exception as e:
            logging.error(f"Failed to recover system: {e}")

    def handle_error(self, error):
        logging.error(f"Error encountered: {error}")
        self.recover_system()

    def save_checkpoint(self, checkpoint_path):
        torch.save(self.state_dict(), checkpoint_path)
        logging.info(f"Checkpoint saved to {checkpoint_path}")

    def load_checkpoint(self, checkpoint_path):
        self.load_state_dict(torch.load(checkpoint_path))
        logging.info(f"Checkpoint loaded from {checkpoint_path}")

    def load_update(self, update_path):
        logging.info(f"Loading update from {update_path}")
        try:
            #save current state as a checkpoint before applying update
            self.save_checkpoint(self.last_checkpoint_path)

            #apply the update
            self.apply_update(update_path)
            logging.info("Update applied successfully.")
        except Exception as e:
            logging.error(f"Failed to apply update: {e}")
            self.rollback_update()

    def apply_update(self, update_path):
        #apply the update by loading new weights from an update file
        new_state_dict = torch.load(update_path)
        self.load_state_dict(new_state_dict)
        logging.info(f"Update applied from {update_path}")

    def rollback_update(self):
        logging.warning("Rolling back to the last checkpoint.")
        if os.path.exists(self.last_checkpoint_path):
            self.load_checkpoint(self.last_checkpoint_path)
            logging.info("Rollback complete.")
        else:
            logging.error("No checkpoint found to roll back to.")

#example usage of the ResilienceModule
#initialize the resilience module
resilience_module = ResilienceModule(input_size=128)

#simulate a training step counter
resilience_module.global_step = 0

#simulate saving a dummy update for testing
dummy_update_path = os.path.join(resilience_module.checkpoint_dir, 'dummy_update.pth')
torch.save(resilience_module.state_dict(), dummy_update_path)

#forward pass and update simulation
for step in range(10):
    resilience_module.global_step = step
    dummy_input = torch.randn(8, 128)
    output = resilience_module(dummy_input)

    #simulate applying an update at step 5
    if step == 5:
        resilience_module.load_update(dummy_update_path)

    if output is not None:
        print(f"Output tensor shape: {output.shape}")
    else:
        print("Output is None due to an error.")

Output tensor shape: torch.Size([8, 1])
Output tensor shape: torch.Size([8, 1])
Output tensor shape: torch.Size([8, 1])
Output tensor shape: torch.Size([8, 1])
Output tensor shape: torch.Size([8, 1])
Output tensor shape: torch.Size([8, 1])
Output tensor shape: torch.Size([8, 1])
Output tensor shape: torch.Size([8, 1])
Output tensor shape: torch.Size([8, 1])
Output tensor shape: torch.Size([8, 1])


In [20]:
#define Yambi model
class Yambi(nn.Module):
    """
    Yambi is an AI model designed to solve abstract reasoning tasks by integrating various cognitive abilities. 
    It consists of the following modules:
    
    1. Perception Module: Processes and understands input grids using convolutional neural networks.
    2. Memory Module: Stores and recalls information about previously seen tasks using LSTM networks.
    3. Creativity Module: Generates creative solutions and explores novel approaches using a latent space.
    4. Reasoning Module: Applies logical reasoning to infer solutions based on perceived information.
    5. Compassionate Module: Analyzes emotional context and adjusts responses accordingly.
    6. Bias Detection and Mitigation Module: Detects and mitigates biases in the reasoning output.
    7. Planning Module: Plans and sequences actions based on inferred solutions.
    8. Attention Module: Focuses on relevant information and enhances processing.
    9. Learning Module: Adapts to new data and refines existing knowledge.
    10. Meta-Reasoning Module: Reflects on the reasoning process and adjusts strategies.
    11. Communication Module: Facilitates interaction and exchange of information.
    12. Integration Module: Integrates outputs from various modules for coherent decision-making.
    13. Resilience Module: Ensures the robustness and stability of the model by adapting to varying conditions, 
    handling disruptions, and maintaining consistent performance through adjustments in the network's parameters and structure.

    These modules work together to perceive, remember, create, reason, empathize, ensure fairness, plan, attend, learn, meta-analyze,
    communicate, integrate, and adapt in solving abstract reasoning tasks.
    """

    def __init__(self, perception_model, memory_model, creativity_model, reasoning_model,
                 compassionate_model, bias_detection_mitigation_model, planning_model,
                 attention_model, learning_model, meta_reasoning_model, communication_model,
                 integration_model, resilience_model):
        super(Yambi, self).__init__()

        self.perception_model = perception_model
        self.memory_model = memory_model
        self.creativity_model = creativity_model
        self.reasoning_model = reasoning_model
        self.compassionate_model = compassionate_model
        self.bias_detection_mitigation_model = bias_detection_mitigation_model
        self.planning_model = planning_model
        self.attention_model = attention_model
        self.learning_model = learning_model
        self.meta_reasoning_model = meta_reasoning_model
        self.communication_model = communication_model
        self.integration_model = integration_model
        self.resilience_model = resilience_model
    
    def forward(self, x):
        # Perception module
        perception_output = self.perception_model(x)
        
        # Memory module
        memory_output = self.memory_model(perception_output)
        
        # Creativity module
        creativity_output = self.creativity_model(memory_output)
        
        # Reasoning module
        reasoning_output = self.reasoning_model(creativity_output)
        
        # Compassionate module
        compassionate_output = self.compassionate_model(reasoning_output)
        
        # Bias Detection and Mitigation module
        bias_detection_output = self.bias_detection_mitigation_model(memory_output)
        
        # Planning module
        planning_output = self.planning_model(reasoning_output)
        
        # Attention module
        attention_output = self.attention_model(reasoning_output)
        
        # Learning module
        learning_output = self.learning_model(reasoning_output)
        
        # Meta-Reasoning module
        meta_reasoning_output = self.meta_reasoning_model(reasoning_output)
        
        # Communication module
        communication_output = self.communication_model(reasoning_output)
        
        # print shapes for debugging
        print("Reasoning output size:", reasoning_output.size())
        print("Compassionate output size:", compassionate_output.size())
        print("Bias Detection output size:", bias_detection_output.size())
        print("Planning output size:", planning_output.size())
        print("Attention output size:", attention_output.size())
        print("Learning output size:", learning_output.size())
        print("Meta-Reasoning output size:", meta_reasoning_output.size())
        print("Communication output size:", communication_output.size())
        
        # Integration module input
        integration_input = torch.cat([
            reasoning_output,
            compassionate_output,
            bias_detection_output,
            planning_output,
            attention_output,
            learning_output,
            meta_reasoning_output,
            communication_output
        ], dim=1)
        
        # print integration input shape for debugging
        print("Integration input size:", integration_input.size())
        
        # Integration module
        integration_output = self.integration_model(integration_input)
        
        # Resilience module
        resilience_output = self.resilience_model(integration_output)
        
        return reasoning_output, compassionate_output, integration_output, resilience_output

#example usage
perception_model = PerceptionModule(in_channels=1)
memory_module = MemoryModule(input_size=512, normalize=True, mean=0.5, std=0.5)
creativity_model = CreativityModule(input_size=512)  
reasoning_model = ReasoningModule(input_size=512, output_size=128)
compassionate_model = CompassionateModule(input_size=128)  
bias_detection_mitigation_model = BiasDetectionMitigationModule(input_size=512)
planning_model = PlanningModule(input_size=128)
attention_model = AttentionModule(input_size=128, output_size=128)  
learning_model = LearningModule(input_size=128)
meta_reasoning_model = MetaReasoningModule(input_size=128, output_size=10)
communication_model = CommunicationModule(input_size=128, output_size=16)

integration_input_size = 128 + 128 + 512 + 128 + 128 + 128 + 10 + 16
integration_model = IntegrationModule(input_size=794)
resilience_model = ResilienceModule(input_size=integration_input_size)

# Instantiate Yambi model
yambi_model = Yambi(
    perception_model, memory_module, creativity_model, reasoning_model,
    compassionate_model, bias_detection_mitigation_model, planning_model,
    attention_model, learning_model, meta_reasoning_model, communication_model,
    integration_model, resilience_model
)

#example forward pass
dummy_input = torch.randn(8, 1, 28, 28)
output = yambi_model(dummy_input)

#print output shapes for verification
print("Reasoning output shape:", output[0].shape)
print("Compassionate output shape:", output[1].shape)
print("Integration output shape:", output[2].shape)
print("Resilience output shape:", output[3].shape)

Reasoning output size: torch.Size([8, 128])
Compassionate output size: torch.Size([8, 128])
Bias Detection output size: torch.Size([8, 128])
Planning output size: torch.Size([8, 128])
Attention output size: torch.Size([8, 128])
Learning output size: torch.Size([8, 128])
Meta-Reasoning output size: torch.Size([8, 10])
Communication output size: torch.Size([8, 16])
Integration input size: torch.Size([8, 794])
Integration input size: torch.Size([8, 794])
Reasoning output shape: torch.Size([8, 128])
Compassionate output shape: torch.Size([8, 128])
Integration output shape: torch.Size([8, 256])
Resilience output shape: torch.Size([8, 1])


In [21]:
import torch.optim as optim

#function to train the model
def train_model(model, dataloader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in dataloader:
            optimizer.zero_grad()
            
            #forward pass
            outputs = model(inputs)
            
            #unpack outputs
            _, _, _, resilience_output = outputs
            
            #print shapes for debugging
            print(f"Resilience output shape: {resilience_output.shape}")
            print(f"Labels shape: {labels.shape}")
            
            #reduce labels to match the resilience output size
            labels = labels.view(labels.size(0), -1).mean(dim=1, keepdim=True)  #flatten and reduce labels to a single value per batch item
            
            #ensure resilience_output and labels have requires_grad=True
            resilience_output = resilience_output.requires_grad_()
            labels = labels.requires_grad_()
            
            #compute loss based on the final output (resilience_output in this case)
            loss = criterion(resilience_output, labels)
            
            #backward pass and optimization
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        average_loss = running_loss / len(dataloader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {average_loss:.4f}")

#function to evaluate the model
def evaluate_model(model, dataloader, criterion):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for inputs, labels in dataloader:
            #forward pass
            outputs = model(inputs)
            
            #unpack outputs
            _, _, _, resilience_output = outputs
            
            #print shapes for debugging
            print(f"Resilience output shape: {resilience_output.shape}")
            print(f"Labels shape: {labels.shape}")
            
            #reduce labels to match the resilience output size
            labels = labels.view(labels.size(0), -1).mean(dim=1, keepdim=True)  #flatten and reduce labels to a single value per batch item
            
            #compute loss based on the final output (resilience_output in this case)
            loss = criterion(resilience_output, labels)
            
            running_loss += loss.item()
    
    average_loss = running_loss / len(dataloader)
    print(f"Evaluation Loss: {average_loss:.4f}")

#example usage
if __name__ == "__main__":
    training_loader = DataLoader(training_dataset, batch_size=8, shuffle=True)
    evaluation_loader = DataLoader(evaluation_dataset, batch_size=8, shuffle=False)
    
    #define criterion and optimizer
    criterion = nn.MSELoss() 
    optimizer = optim.Adam(yambi_model.parameters(), lr=0.001)
    
    #train Yambi model
    train_model(yambi_model, training_loader, criterion, optimizer, num_epochs=10)
    
    #evaluate Yambi model
    evaluate_model(yambi_model, evaluation_loader, criterion)

Reasoning output size: torch.Size([8, 128])
Compassionate output size: torch.Size([8, 128])
Bias Detection output size: torch.Size([8, 128])
Planning output size: torch.Size([8, 128])
Attention output size: torch.Size([8, 128])
Learning output size: torch.Size([8, 128])
Meta-Reasoning output size: torch.Size([8, 10])
Communication output size: torch.Size([8, 16])
Integration input size: torch.Size([8, 794])
Integration input size: torch.Size([8, 794])
Resilience output shape: torch.Size([8, 1])
Labels shape: torch.Size([8, 1, 30, 30])
Reasoning output size: torch.Size([8, 128])
Compassionate output size: torch.Size([8, 128])
Bias Detection output size: torch.Size([8, 128])
Planning output size: torch.Size([8, 128])
Attention output size: torch.Size([8, 128])
Learning output size: torch.Size([8, 128])
Meta-Reasoning output size: torch.Size([8, 10])
Communication output size: torch.Size([8, 16])
Integration input size: torch.Size([8, 794])
Integration input size: torch.Size([8, 794])
Res

In [22]:
#compute the ARC score
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix

def compute_arc_score(model, dataloader):
    model.eval()
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            #forward pass
            outputs = model(inputs)
            
            #unpack outputs
            _, _, _, resilience_output = outputs
            
            #assuming binary classification with a single output neuron per sample
            predictions = (resilience_output > 0.5).int().cpu().numpy()  #binarize the output
            
            #print shapes for debugging
            print(f"Original labels shape: {labels.shape}")
            print(f"Predictions shape: {predictions.shape}")

            #flatten the labels and move to CPU for consistency
            labels = labels.view(labels.size(0), -1).cpu().numpy()  #flatten to (batch_size, 1)
            labels = labels[:, 0]  #assuming the label we care about is the first one
            
            #print shapes after flattening
            print(f"Flattened labels shape: {labels.shape}")
            
            all_labels.append(labels)
            all_predictions.append(predictions)

    #convert lists to numpy arrays
    all_labels = np.concatenate(all_labels).flatten()
    all_predictions = np.concatenate(all_predictions).flatten()

    #print final shapes for debugging
    print(f"Final all_labels shape: {all_labels.shape}")
    print(f"Final all_predictions shape: {all_predictions.shape}")

    #calculate Accuracy
    accuracy = accuracy_score(all_labels, all_predictions)

    #calculate Recall
    recall = recall_score(all_labels, all_predictions, average='macro')

    #calculate Confusion Matrix
    conf_matrix = confusion_matrix(all_labels, all_predictions)

    return accuracy, recall, conf_matrix

#example usage
if __name__ == "__main__":
    evaluation_loader = DataLoader(evaluation_dataset, batch_size=8, shuffle=False)
    
    #compute ARC score
    accuracy, recall, conf_matrix = compute_arc_score(yambi_model, evaluation_loader)
    
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Recall: {recall:.4f}")
    print("Confusion Matrix:")
    print(conf_matrix)

Reasoning output size: torch.Size([8, 128])
Compassionate output size: torch.Size([8, 128])
Bias Detection output size: torch.Size([8, 128])
Planning output size: torch.Size([8, 128])
Attention output size: torch.Size([8, 128])
Learning output size: torch.Size([8, 128])
Meta-Reasoning output size: torch.Size([8, 10])
Communication output size: torch.Size([8, 16])
Integration input size: torch.Size([8, 794])
Integration input size: torch.Size([8, 794])
Original labels shape: torch.Size([8, 1, 30, 30])
Predictions shape: (8, 1)
Flattened labels shape: (8,)
Reasoning output size: torch.Size([8, 128])
Compassionate output size: torch.Size([8, 128])
Bias Detection output size: torch.Size([8, 128])
Planning output size: torch.Size([8, 128])
Attention output size: torch.Size([8, 128])
Learning output size: torch.Size([8, 128])
Meta-Reasoning output size: torch.Size([8, 10])
Communication output size: torch.Size([8, 16])
Integration input size: torch.Size([8, 794])
Integration input size: torc